In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
tf.__version__

'1.12.0'

In [2]:
def plot_loss_history(histories, key='loss'):
    plt.figure(figsize=(8,5))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                       '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

In [3]:
def plot_accuracy_history(histories, key='accuracy'):
    plt.figure(figsize=(8,5))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                       '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

In [4]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import glob
import cv2 
import numpy as np
def load_data():
    IMG_SIZE = 150
    train_path = "./FLOWER/flowers/"
    final_path = "./FLOWER/unlabel-flowers/"
    # TRAINING
    cat = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
    i = 0
    cat_index = 0
    for c in cat:
        file_names = glob.glob(train_path+c+"/*.jpg")
        print(cat_index)
        for f in file_names:
            img = cv2.imread(f, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
#             img = load_img(f)
            if i == 0:
                x_train = [img_to_array(img)]
                y_train = [[cat_index]]
            else:
                x_train = np.append(x_train, [img_to_array(img)], axis=0)
                y_train = np.append(y_train, [[cat_index]], axis=0)
            i += 1
        cat_index += 1
    # FINAL
    final_img_size = 175
    for i in range(1, final_img_size+1):
        img = cv2.imread(final_path+str(i)+'.jpg', cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
#         img = load_img(final_path+str(i)+'.jpg')
        if i == 1:
            x_final = [img_to_array(img)]
        else:
            x_final = np.append(x_final, [img_to_array(img)], axis=0)
            
    return (x_train, y_train), (x_final)

In [5]:
(x_train, y_train), (x_final) = load_data()

0
1
2
3
4


In [6]:
# reshape
x_train = x_train.reshape(x_train.shape[0], 150, 150, 3)
x_final = x_final.reshape(x_final.shape[0], 150, 150, 3)

# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_final = x_final.astype('float32')

In [7]:
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_final', x_final.shape[0])

x_train shape: (4148, 150, 150, 3)
Number of images in x_train 4148
Number of images in x_final 175


In [8]:
# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train)

# normalize -1~1
x_train = (x_train-127.5)/127.5
x_final = (x_final-127.5)/127.5

# validation
x_val = x_train[-1000:]
y_val = y_train[-1000:]
x_train = x_train[:-1000]
y_train = y_train[:-1000]

In [9]:
# set up image augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
)
datagen.fit(x_train)

In [12]:
lrate = 5e-4
epochs = 50
bsize = 128

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.BatchNormalization(axis=-1),
#     tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
#     tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(axis=-1),
#     tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
#     tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.MaxPooling2D((2,2)),
#     tf.keras.layers.Dropout(0.3),
    
#     tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
#     tf.keras.layers.BatchNormalization(axis=-1),
# #     tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
# #     tf.keras.layers.BatchNormalization(axis=-1),
#     tf.keras.layers.MaxPooling2D((2,2)),
# #     tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.summary()
optimizer = tf.keras.optimizers.Adam(lr=lrate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# baseline_history = model.fit(x_train, y_train, epochs=epochs, batch_size=bsize, validation_data=(x_val, y_val))

# train with image augmentation
augmentation_history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=bsize), steps_per_epoch = len(x_train) / bsize, epochs=epochs, validation_data=(x_train, y_train))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 87616)             0         
__________

In [11]:
import csv
predictions = model.predict(x_final)
print('predictions shape:', predictions.shape)

with open('FLOWER_result.csv', mode='w') as result_file:
    writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['ID', 'Class'])
    i = 1
    for p in predictions:
        writer.writerow([i, np.argmax(p)+1])
        i += 1

predictions shape: (175, 5)
